<a href="https://colab.research.google.com/github/Pavel184/NLP_basic_course/blob/Homework_lesson_7/Homework_lesson_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### HOMEWORK_LESSON_7

1. Учим conv сеть для классификации

In [1]:
!pip install pymorphy2

     |████████████████████████████████| 55 kB 3.0 MB/s 
     |████████████████████████████████| 8.2 MB 9.9 MB/s 


In [2]:
!pip install stop-words

  Created wheel for stop-words: filename=stop_words-2018.7.23-py3-none-any.whl size=32912 sha256=2494536622e565c9bd1cdf225b5329ab33ca3399d72a478580bd224c9d215014
  Stored in directory: /root/.cache/pip/wheels/fb/86/b2/277b10b1ce9f73ce15059bf6975d4547cc4ec3feeb651978e9
Successfully built stop-words


In [3]:
import pandas as pd
import numpy as np
import keras
from keras import utils as np_utils
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import TensorBoard 
#from keras.objectives import categorical_crossentropy
from keras.callbacks import EarlyStopping  
from sklearn import model_selection, preprocessing, linear_model
import tensorflow as tf
from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
import re
import nltk
from nltk.tokenize import word_tokenize
nltk.download("punkt")
from nltk.probability import FreqDist
import os


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [4]:
from google.colab import drive

drive.mount('/content/drive')  

Mounted at /content/drive


In [5]:
# Загрузка датасета
df_data = pd.read_excel('/content/drive/My Drive/Colab Notebooks/NLP_basic_course/datasets/отзывы за лето.xls')

In [6]:
# Перевод в тип строка всех отзывов
df_data['Content'] = df_data['Content'].astype(str)

In [7]:
# Просмотр датасета
df_data

,Rating,Content,Date
0,5,It just works!,2017-08-14
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14
2,5,Отлично все,2017-08-14
3,5,Стал зависать на 1% работы антивируса. Дальше ...,2017-08-14
4,5,"Очень удобно, работает быстро.",2017-08-14
...,...,...,...
20654,1,"Ну и шляпа,с роот правами бесполезная прога,ра...",2017-06-01
20655,5,Ок,2017-06-01
20656,4,Доволен,2017-06-01
20657,1,"Песопаснасть, рут ни нужын",2017-06-01


In [8]:
# Для дальнейшей работы берём только сами отзывы
X_col_list = ['Content'] #, 'Date'

In [9]:
# Разделение на тренировочную и тестовую выборки
train_x, test_x, train_y, test_y = model_selection.train_test_split(df_data[X_col_list], df_data['Rating'])

In [10]:
# Очистка текста
sw = set(get_stop_words("ru"))
exclude = set(punctuation)
morpher = MorphAnalyzer()

def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub("\sне", "не", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw]
    return " ".join(txt)

train_x['Content'] = train_x['Content'].apply(preprocess_text)
test_x['Content'] = test_x['Content'].apply(preprocess_text)

In [11]:
# Перевод в нижний регистр
train_corpus = " ".join(train_x["Content"])
train_corpus = train_corpus.lower()

In [12]:
# Токенизация
tokens = word_tokenize(train_corpus)

In [13]:
# Филтрация
tokens_filtered = [word for word in tokens if word.isalnum()]

In [14]:
max_words = 10584
max_len = 40
num_classes = 5

# Training
epochs = 10
batch_size = 512
print_batch_n = 100

In [15]:
dist = FreqDist(tokens_filtered)
tokens_filtered_top = [pair[0] for pair in dist.most_common(max_words-1)]

In [16]:
# Словарь
vocabulary = {v: k for k, v in dict(enumerate(tokens_filtered_top, 1)).items()}

In [17]:
# Функция перевода  последовательность
def text_to_sequence(text, maxlen):
    result = []
    tokens = word_tokenize(text.lower())
    tokens_filtered = [word for word in tokens if word.isalnum()]
    for word in tokens_filtered:
        if word in vocabulary:
            result.append(vocabulary[word])
    padding = [0]*(maxlen-len(result))
    return padding + result[-maxlen:]

In [18]:
#  Тренировочная и тестовая последовательности
x_train = np.asarray([text_to_sequence(text, max_len) for text in train_x['Content']], dtype=np.int32)
x_test = np.asarray([text_to_sequence(text, max_len) for text in test_x['Content']], dtype=np.int32)

In [19]:
# OHE
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
test_y = encoder.fit_transform(test_y)

In [20]:
# Тензоры с метками классов
num_classes = 5
y_train = keras.utils.np_utils.to_categorical(train_y, num_classes)
y_test = keras.utils.np_utils.to_categorical(test_y, num_classes)

In [21]:
'''input_1 = tf.keras.Input(shape=(max_words,max_len))
input_2 = tf.keras.Input(shape=(max_words,max_len))
emb_1 = Embedding(input_dim=max_words, output_dim=128, input_length=max_len)(input_1)'''

'input_1 = tf.keras.Input(shape=(max_words,max_len))\ninput_2 = tf.keras.Input(shape=(max_words,max_len))\nemb_1 = Embedding(input_dim=max_words, output_dim=128, input_length=max_len)(input_1)'

In [22]:
# Слои модели
model_1 = Sequential()
model_1.add(Embedding(input_dim=max_words, output_dim=128, input_length=max_len))
model_1.add(Conv1D(128, 3))
model_1.add(Activation("relu"))
model_1.add(GlobalMaxPool1D())
model_1.add(Dense(10))
model_1.add(Activation("relu"))
model_1.add(Dense(num_classes))
model_1.add(Activation('softmax'))

In [23]:
# Сборка модели
model_1.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [24]:
'''tensorboard=TensorBoard(log_dir='./logs', write_graph=True, write_images=True)
early_stopping=EarlyStopping(monitor='val_loss')  '''

# Тренировка модели
history_1 = model_1.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1)

Epoch 1/10
28/28 [==============================] - 5s 139ms/step - loss: 1.2695 - accuracy: 0.6430 - val_loss: 1.0693 - val_accuracy: 0.7032
Epoch 2/10
28/28 [==============================] - 4s 130ms/step - loss: 0.9162 - accuracy: 0.7179 - val_loss: 0.8142 - val_accuracy: 0.7394
Epoch 3/10
28/28 [==============================] - 4s 129ms/step - loss: 0.6873 - accuracy: 0.7671 - val_loss: 0.6913 - val_accuracy: 0.7529
Epoch 4/10
28/28 [==============================] - 4s 132ms/step - loss: 0.5900 - accuracy: 0.7860 - val_loss: 0.6753 - val_accuracy: 0.7613
Epoch 5/10
28/28 [==============================] - 4s 134ms/step - loss: 0.5262 - accuracy: 0.8100 - val_loss: 0.6756 - val_accuracy: 0.7632
Epoch 6/10
28/28 [==============================] - 4s 133ms/step - loss: 0.4661 - accuracy: 0.8427 - val_loss: 0.6841 - val_accuracy: 0.7665
Epoch 7/10
28/28 [==============================] - 4s 134ms/step - loss: 0.4075 - accuracy: 0.8728 - val_loss: 0.6921 - val_accuracy: 0.7639
Epoch 

In [25]:
score_1 = model_1.evaluate(x_test, y_test, batch_size=batch_size, verbose=1)
print('\n')
print('Test loss:', score_1[0])
print('Test accuracy:', score_1[1])

11/11 [==============================] - 0s 33ms/step - loss: 0.7696 - accuracy: 0.7644


Test loss: 0.7696125507354736
Test accuracy: 0.764375627040863


2. Рассмотреть 2-а варианта сеточек

2.1 Инициализировать tf.keras.layers.Embedding предобученными векторами взять к примеру с https://rusvectores.org/ru/

In [27]:
# Список текстов
texts = []  # list of text samples
df_data_prep = df_data['Content'].apply(preprocess_text)
for text in df_data_prep:
  texts.append(text)

In [28]:
#labels_index = {}  # dictionary mapping label name to numeric id


In [29]:
# Количество отзывов
print('Found %s texts.' % len(texts))

Found 20659 texts.


In [30]:
VALIDATION_SPLIT = 0.3
# Токенизация
tokenizer = Tokenizer(nb_words=max_words)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))
# Подготовка тензоров
data = pad_sequences(sequences, maxlen=max_len)
df_data_y = encoder.fit_transform(df_data['Rating'])
labels = keras.utils.np_utils.to_categorical(df_data_y, num_classes)
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

# split the data into a training set and a validation set
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
nb_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

x_train_weight = data[:-nb_validation_samples]
y_train_weight = labels[:-nb_validation_samples]
x_val_weight = data[-nb_validation_samples:]
y_val_weight = labels[-nb_validation_samples:]

/usr/local/lib/python3.7/dist-packages/keras_preprocessing/text.py:180: UserWarning: The `nb_words` argument in `Tokenizer` has been renamed `num_words`.
  warnings.warn('The `nb_words` argument in `Tokenizer` '


Found 13485 unique tokens.
Shape of data tensor: (20659, 40)
Shape of label tensor: (20659, 5)


In [31]:
# Загрузка предобученных эмбендингов tayga_upos_skipgram_300_2_2019
embeddings_index = {}
f = open(os.path.join('/content/drive/My Drive/Colab Notebooks/NLP_basic_course/datasets/pretrain/', 'model.txt'))
for line in f:
    values = line.split()
    word_and_ner = values[0].split('_')
    word = word_and_ner[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 195436 word vectors.


In [32]:
# Подготовка матрицы весов
EMBEDDING_DIM = 300
embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [33]:
# Слой эмендинг
embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=max_len,
                            trainable=False)

In [34]:
# Слои модели
model_2 = Sequential()
model_2.add(Input(shape=(max_len,), dtype='int32'))
model_2.add(embedding_layer)
model_2.add(Conv1D(128, 3))
model_2.add(Activation("relu"))
model_2.add(GlobalMaxPool1D())
model_2.add(Dense(10))
model_2.add(Activation("relu"))
model_2.add(Dense(num_classes))
model_2.add(Activation('softmax'))

In [35]:
# Компиляция модели
model_2.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [36]:
# Обучение модели
model_2.fit(x_train_weight, y_train_weight, validation_data=(x_val_weight, y_val_weight),
          epochs=10, batch_size=128)

Epoch 1/10
113/113 [==============================] - 7s 62ms/step - loss: 0.8140 - accuracy: 0.7466 - val_loss: 0.7251 - val_accuracy: 0.7583
Epoch 2/10
113/113 [==============================] - 7s 59ms/step - loss: 0.6471 - accuracy: 0.7794 - val_loss: 0.6939 - val_accuracy: 0.7600
Epoch 3/10
113/113 [==============================] - 7s 58ms/step - loss: 0.5756 - accuracy: 0.7986 - val_loss: 0.6674 - val_accuracy: 0.7691
Epoch 4/10
113/113 [==============================] - 7s 59ms/step - loss: 0.5115 - accuracy: 0.8250 - val_loss: 0.6670 - val_accuracy: 0.7647
Epoch 5/10
113/113 [==============================] - 7s 58ms/step - loss: 0.4482 - accuracy: 0.8517 - val_loss: 0.6903 - val_accuracy: 0.7615
Epoch 6/10
113/113 [==============================] - 7s 58ms/step - loss: 0.3967 - accuracy: 0.8750 - val_loss: 0.7289 - val_accuracy: 0.7641
Epoch 7/10
113/113 [==============================] - 7s 58ms/step - loss: 0.3386 - accuracy: 0.8999 - val_loss: 0.7281 - val_accuracy: 0.7554

In [37]:
score_2 = model_2.evaluate(x_val_weight, y_val_weight, batch_size=batch_size, verbose=1)
print('\n')
print('Test loss:', score_2[0])
print('Test accuracy:', score_2[1])

13/13 [==============================] - 1s 74ms/step - loss: 0.8630 - accuracy: 0.7541


Test loss: 0.8629739284515381
Test accuracy: 0.7540745735168457


2.2 Инициализировать слой tf.keras.layers.Embedding по умолчанию (ну то есть вам ничего не делать с весами)

Сравнить две архитектуры с предобученными весами и когда tf.keras.layers.Embedding обучается сразу со всей сеточкой, что получилось лучше

In [42]:
model_3 = Sequential()
model_3.add(Embedding(input_dim=max_words, output_dim=128))
model_3.add(Conv1D(128, 3))
model_3.add(Activation("relu"))
model_3.add(GlobalMaxPool1D())
model_3.add(Dense(10))
model_3.add(Activation("relu"))
model_3.add(Dense(num_classes))
model_3.add(Activation('softmax'))

In [43]:
model_3.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [44]:
tensorboard=TensorBoard(log_dir='./logs', write_graph=True, write_images=True)
early_stopping=EarlyStopping(monitor='val_loss')  


history = model_3.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1)

Epoch 1/10
28/28 [==============================] - 4s 140ms/step - loss: 1.3427 - accuracy: 0.6575 - val_loss: 1.0400 - val_accuracy: 0.7026
Epoch 2/10
28/28 [==============================] - 4s 135ms/step - loss: 0.8970 - accuracy: 0.7154 - val_loss: 0.8158 - val_accuracy: 0.7387
Epoch 3/10
28/28 [==============================] - 4s 135ms/step - loss: 0.7410 - accuracy: 0.7630 - val_loss: 0.7546 - val_accuracy: 0.7548
Epoch 4/10
28/28 [==============================] - 4s 133ms/step - loss: 0.6645 - accuracy: 0.7845 - val_loss: 0.7259 - val_accuracy: 0.7626
Epoch 5/10
28/28 [==============================] - 4s 134ms/step - loss: 0.6001 - accuracy: 0.8054 - val_loss: 0.7134 - val_accuracy: 0.7639
Epoch 6/10
28/28 [==============================] - 4s 137ms/step - loss: 0.5400 - accuracy: 0.8265 - val_loss: 0.7072 - val_accuracy: 0.7697
Epoch 7/10
28/28 [==============================] - 4s 134ms/step - loss: 0.4797 - accuracy: 0.8435 - val_loss: 0.7093 - val_accuracy: 0.7710
Epoch 

In [46]:
score_3 = model_3.evaluate(x_test, y_test, batch_size=batch_size, verbose=1)
print('\n')
print('Test loss:', score_3[0])
print('Test accuracy:', score_3[1])

11/11 [==============================] - 0s 33ms/step - loss: 0.7984 - accuracy: 0.7514


Test loss: 0.7983616590499878
Test accuracy: 0.7514036893844604


Точность модели с предобученным эмбендингом на уровне модели с эмбендингом по умолчанию